In [ ]:
import pandas as pd
import numpy as np
from notebooks.data.feature_scores import calculate_weather_score, calculate_date_score, calculate_team_score, calculate_competition_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Step 1: Load Data

In [ ]:
def load_data():
    tickets_df = pd.read_csv("notebooks/data/complete/ticket_sales.csv")
    weather_df = pd.read_csv("notebooks/data/weather/weather.csv")
    return tickets_df, weather_df

In [ ]:
# Step 2: Data Preprocessing

In [ ]:
def preprocess_data(tickets_df, weather_df):
    # Rename columns for consistency
    weather_df.rename(columns={'date': 'Date'}, inplace=True)
    
    # Convert Date columns to datetime
    weather_df['Date'] = pd.to_datetime(weather_df['Date'])
    tickets_df['Date'] = pd.to_datetime(tickets_df['Date'])
    
    # Merge the dataframes
    merged_df = pd.merge(tickets_df, weather_df, on='Date', how='inner')

    # Drop unnecessary columns
    merged_df.drop(columns=['EventId', 'Venue', 'tsun', 'pres', 'wspd', 'wdir', 'wpgt', 'tmin', 'tmax', 'snow'], inplace=True)

    # Remove duplicates
    merged_df.drop_duplicates(inplace=True)

    # Extract opponent from EventName
    merged_df['Opponent'] = merged_df['EventName'].str.split('vs.').str[1].str.strip()
    merged_df.drop(columns=['EventName'], inplace=True)
    
    return merged_df

In [ ]:
# Step 3: Add Seasonal and Competition Info

In [ ]:
def add_season_and_competition(merged_df):
    def get_season(month):
        if month in [12, 1, 2]:
            return 'Winter'
        elif month in [3, 4, 5]:
            return 'Spring'
        elif month in [6, 7, 8]:
            return 'Summer'
        else:
            return 'Fall'

    # Add season column
    merged_df['Season'] = merged_df['Date'].dt.month.apply(get_season)

    # Clean competition column
    merged_df['Competition'] = merged_df['Competition'].str.replace(r'\s+\d{4}/\d{2}', '', regex=True).str.strip()
    
    return merged_df

In [ ]:
# Step 4: Club Membership and Table Position

In [ ]:
def map_club_membership():
    # Define the membership count for each club
    club_membership = {
        'FC Bayern München': 400000,
        '1. FC Köln': 140000,
        'Leipzig': 40000,
        'VfL Wolfsburg': 21500,
        '1. FC Union Berlin': 70000,
        'Bayer 04 Leverkusen': 70000,
        'Borussia Dortmund': 218000,
        'TSG Hoffenheim': 11000,
        'FC Schalke 04': 190000,
        'Hertha BSC': 60000,
        'SV Werder Bremen': 60000,
        'VfB Stuttgart': 120000,
        'VfL Bochum 1848': 30000,
        'Borussia Mönchengladbach': 102000,
        'FC Augsburg': 25000,
        '1. FSV Mainz 05': 20000,
        'SC Freiburg': 70000,
        'SV Darmstadt 98': 14500,
        '1. FC Heidenheim': 11000,
        'Holstein Kiel': 10000,
        'Eintracht Frankfurt': 150000,

        # International Clubs - estimates or unclear memberships
        'Sporting Clube de Portugal': 160000,
        'Tottenham Hotspur': 25000,
        'Olympique de Marseille': 25000,
        'SSC Napoli': 16000,
        'PFC Le': 3000,  
        'Aberdeen FC': 10000,
        'HJK Helsinki': 5000,
        'PAOK FC': 10000,
        'Royale Union Saint-Gilloise': 5000,
        'FC Viktoria Plzeň': 6000,
        'Eintracht Frankfurt': 125000,
        'FC RFS': 1000,
        'SK Slavia Praha': 12000,
        'Ferencvárosi TC': 8000
    }
    return club_membership

def add_opponent_members(merged_df, club_membership):
    # Add club membership info for opponents
    merged_df['Opponent_Members'] = merged_df['Opponent'].map(club_membership)
    return merged_df

def add_table_position(merged_df):
    top_clubs = ['FC Bayern München', 'Borussia Dortmund', 'Bayer 04 Leverkusen', 'RB Leipzig']
    relegation = ['VfL Bochum 1848', 'SV Darmstadt 98', '1. FC Heidenheim', 'Holstein Kiel']
    
    merged_df['Table_Position_Opponent'] = merged_df['Opponent'].apply(
        lambda x: np.random.randint(1, 5) if x in top_clubs else
                  np.random.randint(15, 19) if x in relegation else
                  np.random.randint(5, 15)
    )
    merged_df['Table_Position_Home'] = np.random.randint(1,14)
    return merged_df

In [ ]:
# Step 5: Generate Form

In [ ]:
def generate_form(opponent):
    top_clubs = ['FC Bayern München', 'Borussia Dortmund', 'Bayer 04 Leverkusen', 'RB Leipzig']
    relegation = ['VfL Bochum 1848', 'SV Darmstadt 98', '1. FC Heidenheim', 'Holstein Kiel']
    # Bias depending on opponent strength
    if opponent in top_clubs:
        probs = [0.6, 0.3, 0.1]  # W, D, L
    elif opponent in relegation:
        probs = [0.2, 0.3, 0.5]  # Weaker teams lose more
    else:
        probs = [0.4, 0.3, 0.3]  # Mid-table balanced

    outcomes = np.random.choice(['W', 'D', 'L'], size=5, p=probs)
    return ''.join(outcomes)

def add_form_column(merged_df):
    merged_df['Form'] = merged_df['Opponent'].apply(generate_form)
    return merged_df

In [ ]:
# Step 6: Filter by Price Category

In [ ]:
def filter_price_categories(merged_df):
    relevant_categories = ['Kat. 1', 'Kat. 2', 'Kat. 3', 'Kat. 4', 'Kat. 5', 'Kat. 6', 'Stehplatz', 'Stehplatz Gast']
    filtered_df = merged_df[merged_df['Price Category'].isin(relevant_categories)].copy()

    # Create dictionaries for each category DataFrame
    category_dfs = {f'Kat. {i}': filtered_df[filtered_df['Price Category'] == f'Kat. {i}'].copy() for i in range(1, 7)}
    category_dfs['Stehplatz'] = filtered_df[filtered_df['Price Category'] == 'Stehplatz'].copy()
    category_dfs['Stehplatz Gast'] = filtered_df[filtered_df['Price Category'] == 'Stehplatz Gast'].copy()

    return category_dfs

In [ ]:
# Example Scoring Method

In [ ]:
def score_calculations(merged_df):
    merged_df['Weather_Score'] = merged_df.apply(lambda row: calculate_weather_score(
        temp=row['tavg'],
        precipitation=row['prcp'],
        season=row['Season'],
    ), axis=1)

    merged_df['Date_Score'] = merged_df.apply(lambda row: calculate_date_score(
        date_str=str(row['Date']),
        time_str=row['Kick-off Time'],
    ), axis=1)

    merged_df['Team_Score'] = merged_df.apply(lambda row: calculate_team_score(
        members=row['Opponent_Members'],
        form_string=row['Form'],
        home_pos=row['Table_Position_Home'], 
        opponent_pos=row['Table_Position_Opponent']
    ), axis=1)
    merged_df['Competition_Score'] = merged_df.apply(lambda row: calculate_competition_score(
        competition=row['Competition'],
    ), axis=1)

    return merged_df

def add_target_clicks(merged_df):
    noise = np.random.normal(0, 100, size=len(merged_df))  # Random noise
    merged_df['Website Traffic'] = merged_df['AVG Price']*20000 + noise
    return merged_df

In [ ]:
# Main Function to Execute Everything

In [ ]:
def main():
    # Load data
    tickets_df, weather_df = load_data()

    # Preprocess data
    merged_df = preprocess_data(tickets_df, weather_df)

    # Add seasonal and competition info
    merged_df = add_season_and_competition(merged_df)

    # Get club membership and add opponent data
    club_membership = map_club_membership()
    merged_df = add_opponent_members(merged_df, club_membership)

    # Add table position
    merged_df = add_table_position(merged_df)

    # Add form column
    merged_df = add_form_column(merged_df)

    # Filter by price category
    category_dfs = filter_price_categories(merged_df)


    kat1scored_df = score_calculations(category_dfs['Kat. 1'])
    kat1scored_df = add_target_clicks(category_dfs['Kat. 1'])

    print(kat1scored_df.head())
    
    kat1scored_df.to_csv('kat1scored.csv', index=False)

    X = kat1scored_df.drop(columns=["Competition", "AVG Price", "Website Traffic", "Booked Tickets", "Price Category", "Date", "Kick-off Time", "tavg", 'prcp', 'Opponent', 'Season', 'Opponent_Members', 'Table_Position_Opponent', 'Table_Position_Home', 'Form' ])  # Drop Target Variable as well as all other features not available at time of prediction 
    Y = kat1scored_df["Website Traffic"]  # Target variable

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    model = RandomForestRegressor()
    model.fit(X_train_scaled, Y_train)

    y_pred = model.predict(X_test_scaled)

    mse = mean_squared_error(Y_test, y_pred)
    r2 = r2_score(Y_test, y_pred)
    mae = mean_absolute_error(Y_test, y_pred)

    print("Mean Absolut:", mae)
    print("Mean Squared Error:", mse)
    print("R-squared Score:", r2)


#Print feature importances
    feature_importance = pd.DataFrame({
    'feature': X.columns,
   'importance': model.feature_importances_
})
    print("\nFeature Importances:")
    print(feature_importance.sort_values('importance', ascending=False))

# Optional: Scatter plot of predicted vs actual Revenue 
    plt.figure(figsize=(10,6))
    plt.scatter(Y_test, y_pred)
    plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r--', lw=2)
    plt.xlabel('Actual Website Trafic')
    plt.ylabel('Predicted Website Trafic')
    plt.title('Website Trafic')
    plt.show()

if __name__ == "__main__":
    main()